Import libraries

In [1]:
import requests
import json
import pandas as pd

Get the URL to access the API

In [2]:
url = 'https://mhw-db.com/monsters'

Retrieve results and assign them to the data variable

In [3]:
data = requests.get(url).json()

Create a dataframe "df" and normalize the 'locations' data

In [4]:
df = pd.json_normalize(data, record_path=['locations'], meta=['name','type','species'], record_prefix='location_')

df.head()

,location_id,location_zoneCount,location_name,name,type,species
0,1,17,Ancient Forest,Aptonoth,small,herbivore
1,1,17,Ancient Forest,Jagras,small,fanged wyvern
2,1,17,Ancient Forest,Mernos,small,wingdrake
3,3,15,Wildspire Waste,Mernos,small,wingdrake
4,1,17,Ancient Forest,Vespoid,small,neopteron


Change the order of columns for better comprehension.

In [5]:
df = df[['name', 'species', 'type', 'location_name', 'location_zoneCount']]

df.head()

,name,species,type,location_name,location_zoneCount
0,Aptonoth,herbivore,small,Ancient Forest,17
1,Jagras,fanged wyvern,small,Ancient Forest,17
2,Mernos,wingdrake,small,Ancient Forest,17
3,Mernos,wingdrake,small,Wildspire Waste,15
4,Vespoid,neopteron,small,Ancient Forest,17


Save the dataset to the csv file

In [7]:
df.to_csv('mhw_monsters.csv')

Now let's create a visualization using plotly + Dash

In [6]:
# import libraries and packages
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

Create a Dash app

In [7]:
app = Dash(__name__)

Aggregate data in the dataframe to see both total counts and categories

In [8]:
agg_df = df.groupby(['location_name', 'type']).agg({'location_zoneCount': 'sum'}).reset_index()
total_counts = agg_df.groupby('location_name')['location_zoneCount'].sum().sort_values(ascending=False)
agg_df['location_name'] = pd.Categorical(agg_df['location_name'], categories=total_counts.index, ordered=True)
agg_df = agg_df.sort_values('location_name', ascending=False)

total_df = df.groupby('species')['location_zoneCount'].sum().reset_index()

Add figure

In [9]:
fig = px.bar(
    agg_df,
    x='location_zoneCount',
    y='location_name',
    color='type',
    orientation='h',
    title='Monsters by Location and Type',
    custom_data=['type']
)

Set up layout

In [10]:
app.layout = html.Div([
    dcc.Graph(id='bar-chart', figure=fig),
    dcc.Graph(id='pie-chart')
])

Add the callback for interactivity

In [11]:
@app.callback(
    Output('pie-chart', 'figure'),
    Input('bar-chart', 'clickData')
)
def update_pie_chart(clickData):
    if clickData:
        location = clickData['points'][0]['y']
        animal_type = clickData['points'][0]['customdata'][0]  # Use customdata to get the type
        filtered_df = df[(df['location_name'] == location) & (df['type'] == animal_type)]
        pie_fig = px.pie(
            filtered_df,
            names='species',
            title=f'Species Distribution in {location} ({animal_type})'
        )
        return pie_fig
    return {}

Run the visualization

In [12]:
if __name__ == '__main__':
    app.run_server(debug=True)